## 7 高级特性
本节，我们将探究一些高级的 Python 特性。你可能会在日常编程中遇到这些特性。虽然许多特性本可以在前面的章节中介绍，但是却没有介绍并不是为了让你在当时避免肝脑涂地。

应该强调的是：本节主题旨在对这些（特性的）思想做基础介绍。你需要查看更高级的材料来了解它们的细节。

### 01 可变参数
本节介绍可变（variadic）参数。有时，可变参数使用 `*args` 和 `**kwargs` 进行表示。
#### __可变位置参数（*args）__
如果一个函数接受任意数量的（位置）参数，那么我们称该函数使用了可变参数（variable arguments）。示例：

In [ ]:
def f(x, *args):
    print(x, args)

f(1)
f(1, 2)
f(1, 2, 3)

额外的参数作为元组进行传递：

In [ ]:
f(1,2,'Hello World')

#### __可变关键字参数（**kwargs）__

一个函数也可以接受任意数量的关键字参数。示例：
```
def f(x, y, **kwargs):
    print(x, y, kwargs)
```
函数调用：
`f(2, 3, flag = True, mode = 'fast', header = 'debug')`
额外的参数作为字典进行传递：
```
def f(x,y,**kwargs):
    # x -> 2
    # y -> 3
    # kwargs -> {'flag': True, 'mode': 'fast', 'header': 'debug'}
```
#### **可变位置参数与可变关键字参数结合使用**
一个函数可以同时接受可变非关键字参数和可变关键字参数。
```
def f(x, *args, **kwargs):
    ...
```
函数调用：
`f(2, 3, flag = True, mode = 'fast', header = 'debug')`
这些参数被分为位置参数和关键字参数两部分。
```
def f(*args, **kwargs):
    # args -> (2, 3)
    # kwargs -> {'flag': True, 'mode': 'fast', 'header': 'debug'}
    ...
```
上述函数接收任意数量的位置参数和关键字参数。当编写包装器（wrappers）或要将参数传递给其他函数时使用。
#### **传递元组和字典**
元组可扩展为可变参数:
```
numbers = (2,3,4)
f(1, *numbers)      # Same as f(1,2,3,4)
```
字典也可以扩展为关键字参数:
```
options = {
    'color' : 'red',
    'delimiter' : ',',
    'width' : 400
}
f(data, **options)
# Same as f(data, color='red', delimiter=',', width=400)
```

### 练习
#### **练习 7.1: 可变参数的简单示例**
尝试定义下列函数：

In [ ]:
def avg(x,*more):
    return float(x + sum(more))/(1 + len(more))
avg(10,11)
avg(3,4,5)
avg(1,2,3,4,5,6)

请注意 `*more` 是如何收集其它所有参数的。　
#### **练习 7.2：将元组和字典作为参数进行传递**
假设你从文件中读取数据，并获得一个元组。例如：

In [ ]:
data = ('GOOG',100, 490.1)

现在，假设你想根据上面的数据创建一个`Stock`对象。如果你直接传`data`，那就行不通了：

In [ ]:
from stock import Stock
s = Stock(data)

这个问题很容易解决，直接使用`*data`即可。试试看：

In [1]:
s = Stock(*data)
s

NameError: name 'Stock' is not defined

如果你拥有的是一个字典，那么你可以使用`**`进行传递。示例：

In [ ]:
data = {
    'name' : 'GOOG',
    'shares' : 100,
    'price' : 490.1
}
s = Stock(**data)
s

#### **练习 7.3：创建实例列表**
在 `report.py` 程序中，我们使用如下代码创建了一个实例列表：
```
def read_portfolio(filename):
    '''
    Read a stock portfolio file into a list of dictionaries with keys
    name, shares, and price.
    '''
    with open(filename) as lines:
        portdicts = fileparse.parse_csv(lines,
                               select=['name','shares','price'],
                               types=[str,int,float])

    portfolio = [ Stock(d['name'], d['shares'], d['price'])
                  for d in portdicts ]
    return Portfolio(portfolio)
```
我们可以改用 Stock(**d) 来简化代码。请完成修改。

#### **练习 7.4：参数传递**
`fileparse.parse_csv()` 函数具有一些选项，用于更改文件分隔符和错误报告。也许你会想把这些选择暴露给上面的 `read_portfolio() `函数。请完成修改：
```
def read_portfolio(filename, **opts):
    '''
    Read a stock portfolio file into a list of dictionaries with keys
    name, shares, and price.
    '''
    with open(filename) as lines:
        portdicts = fileparse.parse_csv(lines,
                                        select=['name','shares','price'],
                                        types=[str,int,float],
                                        **opts)

    portfolio = [ Stock(**d) for d in portdicts ]
    return Portfolio(portfolio)
```
修改完成后，尝试阅读读取一些带有错误的文件：

In [2]:
import report
port = report.read_portfolio('../Data/missing.csv')

第4行: 无法转换 ['MSFT', '', '51.23']
第4行: 原因 invalid literal for int() with base 10: ''
第7行: 无法转换 ['IBM', '', '70.44']
第7行: 原因 invalid literal for int() with base 10: ''


现在尝试隐藏错误：

In [ ]:
import report
port = report.read_portfolio('../Data/missing.csv', silence_errors=True)

### 02 匿名函数和Lambda
#### **再探列表排序**
列表可以使用`sort`方法进行原地（in-place）排序：
```
s = [10,1,7,3]
s.sort() # s = [1,3,7,10]
```
也可以进行降序排列：
```
s = [10,1,7,3]
s.sort(reverse=True) # s = [10,7,3,1]
```
对单个列表排序似乎非常简单，但是，如果对字典列表排序，那么我们怎么做呢？
```
[{'name': 'AA', 'price': 32.2, 'shares': 100},
{'name': 'IBM', 'price': 91.1, 'shares': 50},
{'name': 'CAT', 'price': 83.44, 'shares': 150},
{'name': 'MSFT', 'price': 51.23, 'shares': 200},
{'name': 'GE', 'price': 40.37, 'shares': 95},
{'name': 'MSFT', 'price': 65.1, 'shares': 50},
{'name': 'IBM', 'price': 70.44, 'shares': 100}]
```
通过什么样的规则进行排序呢？

你可以使用键函数（key function)指导排序。键函数是这样的一种函数：接受一个字典并且返回一个用于排序的值。
```
def stock_name(s):
    return s['name']
portfolio.sort(key=stock_name)
```
排序后的结果：
```
[{'name': 'AA', 'price': 32.2, 'shares': 100},
{'name': 'CAT', 'price': 83.44, 'shares': 150},
{'name': 'GE', 'price': 40.37, 'shares': 95},
{'name': 'IBM', 'price': 91.1, 'shares': 50},
{'name': 'IBM', 'price': 70.44, 'shares': 100},
{'name': 'MSFT', 'price': 51.23, 'shares': 200},
{'name': 'MSFT', 'price': 65.1, 'shares': 50}]
```
#### **回调函数**
在上面的示例中，键函数是一个回调函数（callback function)。`sort()` 方法调用了 `stock_name（）`函数，然后 `stock_name（）` 函数返回一个值给 `sort()` 方法。通常，回调函数是一个简短的单行函数，只用于一个操作。程序员经常会要求提供一种快捷方法来指定这种额外的处理。
#### **Lambda 函数**
在之前的排序示例中，我们可以使用 **lambda**， 而不是创建（具名）函数。
```
portfolio.sort(key=lambda s: s['name'])
```
上面的代码创建了一个匿名（unnamed）函数用来计算单个表达式，比初始代码短很多。
```
def stock_name(s):
    return s['name']

portfolio.sort(key=stock_name)

# vs lambda
portfolio.sort(key=lambda s: s['name'])
```
#### **使用lambda**
*    lambda 是受到严格限制的。
*    只允许一个表达式。
*    没有 if，while 之类的语句。
*    多和其它函数一起使用，类似于 `sort()` 方法。
### 练习
读取股票投资组合数据，并将数据转换为列表：

In [22]:
import report
portfolio = list(report.read_portfolio('../Data/portfolio.csv'))
for s in portfolio:
    print(s)

{'name': 'AA', 'shares': 100, 'price': 32.2}
{'name': 'IBM', 'shares': 50, 'price': 91.1}
{'name': 'CAT', 'shares': 150, 'price': 83.44}
{'name': 'MSFT', 'shares': 200, 'price': 51.23}
{'name': 'GE', 'shares': 95, 'price': 40.37}
{'name': 'MSFT', 'shares': 50, 'price': 65.1}
{'name': 'IBM', 'shares': 100, 'price': 70.44}


#### **练习 7.5：对字段进行排序**
请尝试下面的语句，这些语句按股票名称的字母顺序对投资组合数据进行排序。

In [ ]:
def stock_name(s):
    return s.name
portfolio.sort(key=stock_name)
for s in portfolio:
    print(s)

在此部分，`stock_name()` 函数从 `portfolio` 列表的单个条目中提取股票名称。`sort()` 方法使用 `stock_name()` 函数的返回值进行比较并排序。
#### **练习 7.6：使用 lambda 对字段进行排序**
请尝试使用` lambda` 表达式，按股份数目对投资组合数据进行排序：

In [26]:
portfolio.sort(key=lambda s: s.shares])
for s in portfolio:
    print(s)

{'name': 'IBM', 'shares': 50, 'price': 91.1}
{'name': 'MSFT', 'shares': 50, 'price': 65.1}
{'name': 'GE', 'shares': 95, 'price': 40.37}
{'name': 'AA', 'shares': 100, 'price': 32.2}
{'name': 'IBM', 'shares': 100, 'price': 70.44}
{'name': 'CAT', 'shares': 150, 'price': 83.44}
{'name': 'MSFT', 'shares': 200, 'price': 51.23}


尝试按每只股票的价格对投资组合数据进行排序：

In [ ]:
portfolio.sort(key=lambda s: s.price)
for s in portfolio:
    print(s)

注意事项：lambda 是一种非常有用的快捷方式，因为它让你直接在 sort() 方法的调用中定义一个特殊的处理函数，而不必事先定义一个单独的函数。

### 03 返回函数
本节介绍使用函数创建其他函数的思想。
#### **简介**

In [28]:
# 考虑以下函数：
def add(x, y):
    def do_add():
        print('Adding',x,y)
        return x + y
    return do_add

#这是返回其它函数的函数。
a = add(10, 20)
a

<function __main__.add.<locals>.do_add()>

In [29]:
a()

Adding 10 20


30

#### **局部变量**
请观察内部函数时如何引用外部函数定义的变量的。

In [ ]:
def add(x, y):
    def do_add():
        # 'x'和'y'定义在'do_add()'之前
        print('Adding',x,y)
        return x + y
    return do_add

进一步观察会发现，在 add() 函数结束后，这些变量仍然保持存活。

In [30]:
a = add(10, 20)
a()

Adding 10 20


30

#### **闭包**
当内部函数作为输出返回时，这个内部函数就称为闭包（closure)。
```
def add(x, y):
    # do_add是一个闭包
    def do_add():
        print('Adding',x,y)
        return x + y
    return do_add
```
基本特性：闭包保留该函数以后正常运行所需的所有变量的值。可以将闭包视作一个函数，该函数拥有一个额外的环境来保存它所依赖的变量的值。
#### **使用闭包**
虽然闭包是Python的基本特性，但是他们的用法通常很微妙。常见应用：
*    在回调函数中使用。
*    延迟计算。
*    装饰器函数（稍后介绍）。
#### **延迟计算**
考虑下面的例子：
```
def after(seconds, func):
    import time
    time.sleep(seconds)
    func()
```
使用示例：
```
def greeting():
    print('Hello World')

after(30, greeting)
```
函数`after()`会等待30秒，然后调用`greeting()`函数。

闭包附带了其他信息。
```
def add(x, y):
    def do_add():
        print(f'Adding {x} + {y} -> {x+y}')
    return do_add

def after(seconds, func):
    import time
    time.sleep(seconds)
    func()
after(30, add(10, 20))
# 'do_add' 具有如下引用关系：x -> 2 与 y -> 3
```
#### **代码重复**
闭包也可以用作一种避免大量重复的技术。

### 练习
#### **练习 7.7：使用闭包避免重复**
闭包的一个更强大的特性是用于生成重复的代码。让我们回顾练习5.7的代码，该代码中定义了带有类型检查的属性：
```
class Stock:
    def __init__(self, name, shares, price):
        self.name = name
        self.shares = shares
        self.price = price
    ...
    @property
    def shares(self):
        return self._shares

    @shares.setter
    def shares(self, value):
        if not isinstance(value, int):
            raise TypeError('Expected int')
        self._shares = value
    ...
```
与其一遍又一遍地输入代码，不如使用闭包自动创建代码。

请创建 `typedproperty.py` 文件，并把下述代码放到文件中：
```
# typedproperty.py

def typedproperty(name, expected_type):
    private_name = '_' + name
    @property
    def prop(self):
        return getattr(self, private_name)

    @prop.setter
    def prop(self, value):
        if not isinstance(value, expected_type):
            raise TypeError(f'Expected {expected_type}')
        setattr(self, private_name, value)

    return prop
```
现在，通过定义下面这样的类来尝试一下：
```
from typedproperty import typedproperty

class Stock:
    name = typedproperty('name', str)
    shares = typedproperty('shares', int)
    price = typedproperty('price', float)

    def __init__(self, name, shares, price):
        self.name = name
        self.shares = shares
        self.price = price
```
请尝试创建一个实例，并验证类型检查是否有效：

In [34]:
from stock import Stock
s = Stock('GOOG', 100, 490.1)
s.name
s.shares = '100'

TypeError: Expected <class 'int'>

#### **练习 7.8：简化函数调用**
在上面示例中，用户可能会发现调用诸如 `typedproperty('shares', int)` 这样的方法稍微有点冗长 ——尤其是多次重复调用的时候。请将以下定义添加到 `typedproperty.py` 文件中。
```
String = lambda name: typedproperty(name, str)
Integer = lambda name: typedproperty(name, int)
Float = lambda name: typedproperty(name, float)
```
现在，请重新编写 `Stock` 类以使用以下函数：
```
class Stock:
    name = String('name')
    shares = Integer('shares')
    price = Float('price')

    def __init__(self, name, shares, price):
        self.name = name
        self.shares = shares
        self.price = price
```
啊，好一点了。这里的要点是：闭包和 `lambda` 常用于简化代码，并消除令人讨厌的代码重复。这通常很不错。

#### **练习 7.9：付诸实践**
请重新编写 `stock.py `文件中的 `Stock` 类，以便使用上面展示的类型化特性（typed properties）。

### 04 函数装饰器
本节介绍装饰器（decorator）。因为这是一个高级主题，所以我们只做简单介绍。
#### **日志示例**
考虑这样一个函数：

In [35]:
def add(x, y):
    return x + y

考虑给'add(x,y)`函数添加日志功能：

In [ ]:
def add(x, y):
    print(f'Adding {x} + {y} -> {x+y}')
    return x + y

也带有日志功能的`sub(x,y)`函数：

In [ ]:
def sub(x, y):
    print(f'Subtracting {x} - {y} -> {x-y}')
    return x - y

#### **观察**
观察： 这是一种重复。

在有大量重复代码的地方编写程序通常很烦人。这些代码不仅写起来枯燥，维护起来也很麻烦。尤其是你决定更改其工作方式的时候（例如，可能是另一种类型的日志记录）。
#### **记录日志的代码**
也许你可以创建一个添加了日志功能的函数。例如包装器（wrapper）：

In [36]:
def logged(func):
    def wrapper(*args, **kwargs):
        print(f'Calling {func.__name__}')
        return func(*args, **kwargs)
    return wrapper

使用该函数：

In [37]:
def add(x, y):
    return x + y
logged_add = logged(add)

当调用`logged`返回的函数时会发生什么？

In [38]:
logged_add(3, 4)

Calling add


7

注意事项：`logged()`函数创建了一个闭包，并作为结果返回。
#### **使用装饰器**
在Python中，在函数中使用装饰器非常常见。因为如此普遍，所以有一个特殊的语法：

In [39]:
def add(x, y):
    return x + y
add = logged(add)

# 特殊语法
@ logged
def add(x, y):
    return x + y

该特殊语法执行与上面完全相同的确切步骤。装饰器只是一种新语法，用于装饰函数。
#### **说明**
对于装饰器而言，还有许多比这里展示的更微妙的细节，例如，在类里面使用装饰器，或者对同一个函数使用多个装饰器。不过，这里的例子已经很好地说明了如何使用它们。一般而言，它是对出现在各种函数定义中的重复代码的响应。装饰器可以将重复代码移至中心定义。

### 练习
#### **练习 7.10：计时装饰器**
如果你定义了一个函数，那么函数的名称和函数所属模块的名称会分别存储到 `__name__` 和 `__module__`属性中。示例：

In [40]:
def add(x, y):
    return x + y
add.__name__

'add'

In [41]:
add.__module__

'__main__'

请创建 `timethis.py` 文件，并在文件中编写 `timethis(func)` 函数。`timethis(func)` 函数包装一个具有额外逻辑层的函数，逻辑层打印出函数执行所需要的事件。为此，你将在函数中添加如下计时调用。
```
start = time.time()
r = func(*args,**kwargs)
end = time.time()
print('%s.%s: %f' % (func.__module__, func.__name__, end-start))
```
（`timethis(func)`）装饰器工作方式示例：
```
>>> from timethis import timethis
>>> @timethis
def countdown(n):
        while n > 0:
             n -= 1

>>> countdown(10000000)
__main__.countdown : 0.076562
>>>
```
讨论：`@timethis` 装饰器可以放在任何函数的前面，即你应该把装饰器用作性能调优（performance tuning）的诊断工具。

In [1]:
from timethis import timethis
@timethis
def countdown(n):
    while n > 0:
        n -= 1
countdown(10000000)

__main__.countdown 运行时间: 0.329256s


### 05 装饰方法
本节讨论一些与方法定义结合使用的内置装饰器。
#### **预定义的装饰器**
在类定义中，有许多预定义的装饰器用于指定特殊方法。例如，`@staticmethod` 和 `@classmethod`、`@property`。
```
class Foo:
    def bar(self,a):
        ...

    @staticmethod
    def spam(a):
        ...

    @classmethod
    def grok(cls,a):
        ...

    @property
    def name(self):
        ...
```
让我们逐个查看吧。
#### **静态方法**
`@staticmethod` 用于定义所谓的静态类方法（ static class method，来自于 C++/Java）。静态方法是一个函数，这个函数是类的一部分，但不是在实例上进行操作。
```
class Foo(object):
    @staticmethod
    def bar(x):
        print('x =', x)

>>> Foo.bar(2)
x=2
>>>
```
静态方法有时用于实现类的内部支持代码，例如，用于帮助管理已创建的实例（内存管理，系统资源，持久化，锁等等）。有时也用于某些设计模式（这里暂不讨论）。
#### **类方法**
`@classmethod` 用于定义类方法（class methods）。类方法是一种将 类 对象而不是实例作为第一个参数的方法。
```
class Foo:
    def bar(self):
        print(self)

    @classmethod
    def spam(cls):
        print(cls)

>>> f = Foo()
>>> f.bar()
<__main__.Foo object at 0x971690>   # The instance `f`
>>> Foo.spam()
<class '__main__.Foo'>              # The class `Foo`
>>>
```
类方法常用作定义替代构造函数（constructor）的工具。
```
import time
class Date:
    def __init__(self,year,month,day):
        self.year = year
        self.month = month
        self.day = day

    @classmethod
    def today(cls):
        # Notice how the class is passed as an argument
        tm = time.localtime()
        # And used to create a new instance
        return cls(tm.tm_year, tm.tm_mon, tm.tm_mday)

d = Date.today()
```
类方法可以和继承等特性一起使用以解决棘手的问题。
```
class Date:
    def __init__(self, year, month, day):
        self.year = year
        self.month = month
        self.day = day

    @classmethod
    def today(cls):
        # Gets the correct class (e.g. `NewDate`)
        tm = time.localtime()
        return cls(tm.tm_year, tm.tm_mon, tm.tm_mday)

class NewDate(Date):
    print('我是NewDate，父类是', Date)

d = NewDate.today()
```

In [4]:
import time

class Date:
    def __init__(self, year, month, day):
        self.year = year
        self.month = month
        self.day = day

    @classmethod
    def today(cls):
        # Gets the correct class (e.g. `NewDate`)
        tm = time.localtime()
        return cls(tm.tm_year, tm.tm_mon, tm.tm_mday)

class NewDate(Date):
    print('我是NewDate，父类是', Date)

d = NewDate.today()

我是NewDate，父类是 <class '__main__.Date'>


### 练习
#### **练习 7.11：实践中的类方法**
在 `report.py` 和 `portfolio.py` 文件中， `Portfolio` 类的创建稍微有点混乱。例如，`report.py` 程序具有如下代码：
```
def read_portfolio(filename, **opts):
    '''
    Read a stock portfolio file into a list of dictionaries with keys
    name, shares, and price.
    '''
    with open(filename) as lines:
        portdicts = fileparse.parse_csv(lines,
                                        select=['name','shares','price'],
                                        types=[str,int,float],
                                        **opts)

    portfolio = [ Stock(**d) for d in portdicts ]
    return Portfolio(portfolio)
```
且 portfolio.py 文件中定义的 Portfolio 具有一个奇怪的初始化：
```
class Portfolio:
    def __init__(self, holdings):
        self.holdings = holdings
    ...
```
坦白说，因为代码分散在各文件中，所以责任链稍微有点混乱。如果 `Portfolio` 类应该包含 `Stock` 类的实例列表，那么你应该修改该类以使其更清晰。示例：
```
# portfolio.py

import stock

class Portfolio:
    def __init__(self):
        self.holdings = []

    def append(self, holding):
        if not isinstance(holding, stock.Stock):
            raise TypeError('Expected a Stock instance')
        self.holdings.append(holding)
    ...
```
如果想要从 CSV 文件中读取投资组合数据，那么你也许应该为此创建一个类方法：
```
# portfolio.py

import fileparse
import stock

class Portfolio:
    def __init__(self):
        self.holdings = []

    def append(self, holding):
        if not isinstance(holding, stock.Stock):
            raise TypeError('Expected a Stock instance')
        self.holdings.append(holding)

    @classmethod
    def from_csv(cls, lines, **opts):
        self = cls()
        portdicts = fileparse.parse_csv(lines,
                                        select=['name','shares','price'],
                                        types=[str,int,float],
                                        **opts)

        for d in portdicts:
            self.append(stock.Stock(**d))

        return self
```
要使用新的 Portfolio 类，你可以这样编写代码：
```
>>> from portfolio import Portfolio
>>> with open('Data/portfolio.csv') as lines:
...     port = Portfolio.from_csv(lines)
...
>>>
```
请对 `Portfolio` 类进行修改，然后修改 `report.py` 的代码以使用类方法。